# Introduction to Altair

> *This introduction is based on the excellent [data visualization curriculum](https://github.com/uwdata/visualization-curriculum) by Jeffrey Heer, Dominik Moritz, Jake VanderPlas, and Brock Craft and adapted to Altair version 5*

- [Altair](https://altair-viz.github.io/) is a declarative statistical visualization library for Python. 

- *declarative*: a high-level specification of *what* you want the visualization to include

- Altair is based on [Vega-Lite](https://vega.github.io/vega-lite/), a high-level grammar of interactive graphics. 

## Imports

- To start, we must import the necessary libraries
  - Pandas for data frames 
  - Altair for visualization 
  - We use `alt.renderers.enable('mimetype')` to render the output in VS Code

  
  **Important note: Make sure to use Altair version 5** (install [env-dst-pip.yml](https://github.com/kirenz/environments))

In [1]:
import pandas as pd
import altair as alt
alt.renderers.enable('mimetype')

alt.__version__

'5.0.0rc1'

## Renderers

- Depending on your environment, you may need to specify a [renderer](https://altair-viz.github.io/user_guide/display_frontends.html) for Altair. 
- If you are using __JupyterLab__, __Jupyter Notebook__, or __Google Colab__ with a live Internet connection you should not need to do anything. 
- Otherwise, please read the documentation for [Displaying Altair Charts](https://altair-viz.github.io/user_guide/display_frontends.html).

## Data

- Data in Altair is built around the Pandas data frame, which consists of a set of named data *columns*. 

- We will also regularly refer to data columns as data *fields*.

<!--
When using Altair, datasets are commonly provided as data frames. Alternatively, Altair can also accept a URL to load a network-accessible dataset. As we will see, the named columns of the data frame are an essential piece of plotting with Altair.
-->

- We will often use datasets from the [vega-datasets](https://github.com/vega/vega-datasets) repository. 

- Some of these datasets are directly available as Pandas data frames:

In [2]:
from vega_datasets import data  # import vega_datasets
cars = data.cars()              # load cars data as a Pandas data frame
cars.head()                     # display the first five rows

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


- Datasets in the vega-datasets collection can also be accessed via URLs:

In [3]:
data.cars.url

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json'

- Dataset URLs can be passed directly to Altair (for supported formats like JSON and [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)), or loaded into a Pandas data frame like so:

In [4]:
pd.read_json(data.cars.url).head() # load JSON data into a data frame

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA


For more information about data frames - and some useful transformations to prepare Pandas data frames for plotting with Altair! - see the [Specifying Data with Altair documentation](https://altair-viz.github.io/user_guide/data.html).

### Weather Data

- Statistical visualization in Altair begins with ["tidy"](http://vita.had.co.nz/papers/tidy-data.html) data frames. 
- Here, we'll start by creating a simple data frame (`df`) containing the average precipitation 🌧️ (`precip`) for a given `city` and `month` :

In [5]:
df = pd.DataFrame({
    'city': ['Seattle', 'Seattle', 'Seattle', 'New York', 'New York', 'New York', 'Chicago', 'Chicago', 'Chicago'],
    'month': ['Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec', 'Apr', 'Aug', 'Dec'],
    'precip': [2.68, 0.87, 5.31, 3.94, 4.13, 3.58, 3.62, 3.98, 2.56]
})

df

,city,month,precip
0,Seattle,Apr,2.68
1,Seattle,Aug,0.87
2,Seattle,Dec,5.31
3,New York,Apr,3.94
4,New York,Aug,4.13
5,New York,Dec,3.58
6,Chicago,Apr,3.62
7,Chicago,Aug,3.98
8,Chicago,Dec,2.56


## The Chart Object

- The fundamental object in Altair is the `Chart`, which takes a data frame as a single argument:

In [6]:
chart = alt.Chart(df)

- So far, we have defined the `Chart` object and passed it the simple data frame we generated above. 
- We have not yet told the chart to *do* anything with the data.

## Marks

- With a chart object in hand, we can now specify how we would like the data to be visualized. 

- We first indicate what kind of graphical **mark** (geometric shape) we want to use to represent the data. 

- We can set the `mark` attribute of the chart object using the the `Chart.mark_*` methods.

- For example, we can show the data as a point using `Chart.mark_point()`:

In [7]:
alt.Chart(df).mark_point()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- Here the rendering consists of one point per row in the dataset, all plotted on top of each other, since we have not yet specified positions for these points.

## Encodings

- To visually separate the points, we can map various **encoding channels**, or **channels** for short, to fields in the dataset. 

- For example, we could *encode* the field `city` of the data using the `y` channel, which represents the y-axis position of the points. 
- To specify this, use the `encode` method:

In [8]:
alt.Chart(df).mark_point().encode(
  y='city',
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- The `encode()` method builds a key-value mapping between encoding channels (such as `x`, `y`, `color`, `shape`, `size`, *etc.*) to fields in the dataset, accessed by field name. 

- For Pandas data frames, Altair automatically determines an appropriate data type for the mapped column 
- Which in this case is the *nominal* type, indicating unordered, categorical values.

- Though we've now separated the data by one attribute, we still have multiple points overlapping within each category. 

- Let's further separate these by adding an `x` encoding channel, mapped to the `'precip'` field:

In [9]:
alt.Chart(df).mark_point().encode(
    x='precip',
    y='city'
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- _Seattle exhibits both the least-rainiest and most-rainiest months!_

- The data type of the `'precip'` field is again automatically inferred by Altair, and this time is treated as a *quantitative* type (that is, a real-valued number). 

- We see that grid lines and appropriate axis titles are automatically added as well.

- Above we have specified key-value pairs using keyword arguments (`x='precip'`). 

- In addition, Altair provides construction methods for encoding definitions 
- We use the syntax `alt.X('precip')`. 
- This alternative is useful for providing more parameters to an encoding, as we will see later in this notebook.

In [10]:
alt.Chart(df).mark_point().encode(
    alt.X('precip'),
    alt.Y('city')
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- The two styles of specifying encodings can be interleaved: 
  - `x='precip', alt.Y('city')` is also a valid input to the `encode` function.

#### Data types

- In the examples, the data type for each field is inferred automatically based on its type within the Pandas data frame. 

- We can also explicitly indicate the data type to Altair by annotating the field name:

- `:N` indicates a *nominal* type (unordered, categorical data),

- `:O` indicates an *ordinal* type (rank-ordered data),

- `:Q` indicates a *quantitative* type (numerical data with meaningful magnitudes)

- `:T` indicates a *temporal* type (date/time data)

For example, `alt.X('precip:N')`.

Explicit annotation of data types is necessary when data is loaded from an external URL directly by Vega-Lite (skipping Pandas entirely), or when we wish to use a type that differs from the type that was automatically inferred.

What do you think will happen to our chart above if we treat `precip` as a nominal or ordinal variable, rather than a quantitative variable? _Modify the code above and find out!_

## Data Transformation: Aggregation

- To allow for more flexibility in how data are visualized, Altair has a built-in syntax for *aggregation* of data. 
- For example, we can compute the average of all values by specifying an aggregation function along with the field name:

In [11]:
alt.Chart(df).mark_point().encode(
    x='average(precip)',
    y='city'
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


Now within each x-axis category, we see a single point reflecting the *average* of the values within that category.

_Does Seattle really have the lowest average precipitation of these cities? (It does!) Still, how might this plot mislead? Which months are included? What counts as precipitation?_

- Altair supports a variety of aggregation functions, including 
  - `count`, 
  - `min` (minimum), 
  - `max` (maximum), 
  - `average`, 
  - `median`, and 
  - `stdev` (standard deviation). 
 

 
 In a later notebook, we will take a tour of data transformations, including aggregation, sorting, filtering, and creation of new derived fields using calculation formulas.

## Changing the Mark Type

- Let's say we want to represent our aggregated values using rectangular bars rather than circular points. 
- We can do this by replacing `Chart.mark_point` with `Chart.mark_bar`:

In [12]:
alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city'
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- Because the nominal field `a` is mapped to the `y`-axis, the result is a horizontal bar chart. To get a vertical bar chart, we can simply swap the `x` and `y` keywords:

In [13]:
alt.Chart(df).mark_bar().encode(
    x='city',
    y='average(precip)'
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## Customizing a Visualization

- By default Altair / Vega-Lite make some choices about properties of the visualization 
- but these can be changed using methods to customize the look of the visualization. 
- For example, we can specify the axis titles using the `axis` attribute of channel classes, we can modify scale properties using the `scale` attribute, and we can specify the color of the marking by setting the `color` keyword of the `Chart.mark_*` methods to any valid [CSS color string](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value):

In [14]:
alt.Chart(df).mark_point(color='firebrick').encode(
  alt.X('precip').scale(type='log').title('Log-Scaled Values'),
  alt.Y('city').title('Category')
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- A subsequent module will explore the various options available for scales, axes, and legends to create customized charts.

## Multiple Views

As we've seen above, the Altair `Chart` object represents a plot with a single mark type. What about more complicated diagrams, involving multiple charts or layers? Using a set of *view composition* operators, Altair can take multiple chart definitions and combine them to create more complex views.

As a starting point, let's plot the cars dataset in a line chart showing the average mileage by the year of manufacture:

In [15]:
alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- To augment this plot, we might like to add `circle` marks for each averaged data point. 

(The `circle` mark is just a convenient shorthand for `point` marks that used filled circles.)

- We can start by defining each chart separately: 
- first a line plot, then a scatter plot. 
- We can then use the `layer` operator to combine the two into a layered chart. 
- Here we use the shorthand `+` (plus) operator to invoke layering:

In [16]:
line = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

point = alt.Chart(cars).mark_circle().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

line + point

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- We can also create this chart by *reusing* and *modifying* a previous chart definition!

Rather than completely re-write a chart, we can start with the line chart, then invoke the `mark_point` method to generate a new chart definition with a different mark type:

In [17]:
mpg = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)

mpg + mpg.mark_circle()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


<em>(The need to place points on lines is so common, the `line` mark also includes a shorthand to generate a new layer for you. Trying adding the argument `point=True` to the `mark_line` method!)</em>

In [18]:
alt.Chart(cars).mark_line(point=True).encode(
    alt.X('Year'),
    alt.Y('average(Miles_per_Gallon)')
)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- Now, what if we'd like to see this chart alongside other plots, such as the average horsepower over time?

- We can use *concatenation* operators to place multiple charts side-by-side, either vertically or horizontally. 

- Here, we'll use the `|` (pipe) operator to perform horizontal concatenation of two charts:

In [19]:
hp = alt.Chart(cars).mark_line().encode(
    alt.X('Year'),
    alt.Y('average(Horsepower)')
)

(mpg + mpg.mark_circle()) | (hp + hp.mark_circle())

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


_We can see that, in this dataset, over the 1970s and early '80s the average fuel efficiency improved while the average horsepower decreased._

A later notebook will focus on *view composition*, including not only layering and concatenation, but also the `facet` operator for splitting data into sub-plots and the `repeat` operator to concisely generate concatenated charts from a template.

## Interactivity

- In addition to basic plotting and view composition, one of Altair and Vega-Lite's most exciting features is its support for interaction.

- To create a simple interactive plot that supports panning and zooming, we can invoke the `interactive()` method of the `Chart` object. 


- In the chart below, click and drag to *pan* or use the scroll wheel to *zoom*:

In [20]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
).interactive()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- To provide more details upon mouse hover, we can use the `tooltip` encoding channel:

In [21]:
alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
    tooltip=['Name', 'Origin'] # show Name and Origin in a tooltip
).interactive()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


- For more complex interactions, such as linked charts and cross-filtering, Altair provides a *selection* abstraction for defining interactive selections and then binding them to components of a chart. 

- We will cover this is in detail in a later notebook.

## Publishing a Visualization

- Once you have visualized your data, perhaps you would like to publish it somewhere on the web. 

- This can be done straightforwardly using the [vega-embed JavaScript package](https://github.com/vega/vega-embed). 

- A simple example of a stand-alone HTML document can be generated for any chart using the `Chart.save` method:

In [22]:
chart = alt.Chart(df).mark_bar().encode(
    x='average(precip)',
    y='city',
)
chart.save('chart.html')


The basic HTML template produces output that looks like this, where the JSON specification for your plot produced by `Chart.to_json` should be stored in the `spec` JavaScript variable:

```html
<!DOCTYPE html>
<html>
  <head>
    <script src="https://cdn.jsdelivr.net/npm/vega@5"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-lite@4"></script>
    <script src="https://cdn.jsdelivr.net/npm/vega-embed@6"></script>
  </head>
  <body>
  <div id="vis"></div>
  <script>
    (function(vegaEmbed) {
      var spec = {}; /* JSON output for your chart's specification */
      var embedOpt = {"mode": "vega-lite"}; /* Options for the embedding */

      function showError(el, error){
          el.innerHTML = ('<div style="color:red;">'
                          + '<p>JavaScript Error: ' + error.message + '</p>'
                          + "<p>This usually means there's a typo in your chart specification. "
                          + "See the javascript console for the full traceback.</p>"
                          + '</div>');
          throw error;
      }
      const el = document.getElementById('vis');
      vegaEmbed("#vis", spec, embedOpt)
        .catch(error => showError(el, error));
    })(vegaEmbed);
  </script>
</body>
</html>
 ```

The `Chart.save` method provides a convenient way to save such HTML output to file. For more information on embedding Altair/Vega-Lite, see the [documentation of the vega-embed project](https://github.com/vega/vega-embed).

## Next Steps

- 🎉 Hooray, you've completed the introduction to Altair! 

- In the next notebook, we will dive deeper into creating visualizations using Altair's model of data types, graphical marks, and visual encoding channels.